# Prepare geodata from the [Australian Bureau of Statistics](https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/main-structure-and-greater-capital-city-statistical-areas/statistical-area-level-1) and Melbourne Housing Data

[Quote](https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/main-structure-and-greater-capital-city-statistical-areas/statistical-area-level-1)

Statistical Areas Level 1 (SA1s) are geographic areas built from whole Mesh Blocks. Whole SA1s aggregate to form Statistical Areas Level 2 (SA2s). SA1s are designed to maximise the geographic detail available for Census of Population and Housing data. However, limited Census of Population and Housing data may also be available at the Mesh Block level for ASGS Edition 3.

* First, the data is filtered to the state of Victoria
    * The SA data is filtered to anything contains `Melbourne` in either of SA2, SA3 or SA4.
    * The Greater Capital City of `Melbourne` 

It seems that the set
```python
melbourne_sa = victoria.loc[victoria.SA2_NAME21.str.contains('Melbourne') | victoria.SA3_NAME21.str.contains('Melbourne') | victoria.SA4_NAME21.str.contains('Melbourne')]
```
is contained in the set
```python
melbourne_gcc = victoria.loc[victoria.GCC_NAME21.str.contains('Melbourne')]
```

Still, both datasets are stored back.

In [1]:
import os, sys
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


import descartes
import geopandas as gpd

from shapely.geometry import Point, Polygon

filepath = os.path.abspath('')
utilpath = f'{filepath}/../util/'
sys.path.append(utilpath)
data_dir = f'{filepath}/../data/'

# time intense
sua = gpd.read_file(f'{data_dir}/2021_AUST_SHP_GDA2020/SUA_2021_AUST_GDA2020_SHP/SUA_2021_AUST_GDA2020.shp')
sa1 = gpd.read_file(f'{data_dir}/2021_AUST_SHP_GDA2020/SA1_2021_AUST_SHP_GDA2020/SA1_2021_AUST_GDA2020.shp')

victoria = sa1.loc[sa1['STE_NAME21'] == 'Victoria']
melbourne_sua = sua.loc[sua['SUA_NAME21'] == 'Melbourne']

melbourne_sa = victoria.loc[
    victoria.SA2_NAME21.str.contains('Melbourne') | 
    victoria.SA3_NAME21.str.contains('Melbourne') | 
    victoria.SA4_NAME21.str.contains('Melbourne')
]
melbourne_gcc = victoria.loc[victoria.GCC_NAME21.str.contains('Melbourne')]

geometry_victoria = victoria['geometry']
geometry_melbourne_sua = melbourne_sua['geometry']
geometry_melbourne_sa = melbourne_sa['geometry']
geometry_melbourne_gcc = melbourne_gcc['geometry']

In [2]:
victoria.to_file(f'{data_dir}/prepared/victoria.shp', driver='ESRI Shapefile')
melbourne_sua.to_file(f'{data_dir}/prepared/melbourne_sua.shp', driver='ESRI Shapefile')
melbourne_sa.to_file(f'{data_dir}/prepared/melbourne_sa.shp', driver='ESRI Shapefile')
melbourne_gcc.to_file(f'{data_dir}/prepared/melbourne_gcc.shp', driver='ESRI Shapefile')

geometry_victoria.to_file(f'{data_dir}/prepared/geometry_victoria.shp', driver='ESRI Shapefile')
geometry_melbourne_sua.to_file(f'{data_dir}/prepared/geometry_melbourne_sua.shp', driver='ESRI Shapefile')
geometry_melbourne_sa.to_file(f'{data_dir}/prepared/geometry_melbourne_sa.shp', driver='ESRI Shapefile')
geometry_melbourne_gcc.to_file(f'{data_dir}/prepared/geometry_melbourne_gcc.shp', driver='ESRI Shapefile')

In [3]:
# Finally, we show that the GCC_NAME21 already contain the SA{2,3,4}_NAME21 data
melbourne_sa_gcc = victoria.loc[
    victoria.SA2_NAME21.str.contains('Melbourne') | 
    victoria.SA3_NAME21.str.contains('Melbourne') | 
    victoria.SA4_NAME21.str.contains('Melbourne') | 
    victoria.GCC_NAME21.str.contains('Melbourne')
]
assert melbourne_gcc.shape[0] == melbourne_sa_gcc.shape[0]
assert melbourne_sa.shape[0] < melbourne_sa_gcc.shape[0]

In [4]:
# Since the .csv format does not support dtypes, the following feature columns
# are actually `int`s but load as dtype `float64`.
# Therefore, they are converted to `int64`s.
melb_data = pd.read_csv(f'{data_dir}/melb_data_orig.csv', dtype={
    'Rooms': int,
    'Bedroom2': int,
    'Bathroom': int,
    'Propertycount': int,
})
# Must use pd.Int64Dtype() for the following feature columns as they contain `nan`s which 
# cannot be converted to ordinary `int64`s.
# However, the problem then is that this cannot be stored into .hdf5 format.
# Having the following lines commented out leaves the dtypes for those column to `float64`

# melb_data['Car'] = melb_data['Car'].astype(pd.Int64Dtype())
# melb_data['YearBuilt'] = melb_data['YearBuilt'].astype(pd.Int64Dtype())

We change the format of `Date` to `pandas._libs.tslibs.timestamps.Timestamp` so that is becomes easier to sort or derive other representations, e. g.
```python
melb_data['Month'] = melb_data.Date.apply(lambda d: d.month)
```

The derivation of new numerical features like `Month` or `Time` from `Date` can capture seasonal trends and longterm effects like inflation.

In [5]:
melb_data.Date = pd.to_datetime(melb_data.Date, format=r'%d/%m/%Y')
melb_data['Month'] = melb_data.Date.apply(lambda d: d.month)
melb_data.head(2)

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Latitude,Longitude,Regionname,Propertycount,Month
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,2016-12-03,2.5,3067.0,...,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019,12
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,2016-02-04,2.5,3067.0,...,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019,2


In [6]:
melb_data.to_csv(f'{data_dir}/melb_data.csv')
melb_data.to_hdf(f'{data_dir}/melb_data.h5', key='melb_data', mode='w')

/var/folders/7f/gh6c5k2j1lb85_805l_29r580000gn/T/ipykernel_64589/1085616258.py:2: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->Index(['Suburb', 'Address', 'Type', 'Method', 'SellerG', 'CouncilArea',
       'Regionname'],
      dtype='object')]

  melb_data.to_hdf(f'{data_dir}/melb_data.h5', key='melb_data', mode='w')


In [7]:
melb_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Suburb         13580 non-null  object        
 1   Address        13580 non-null  object        
 2   Rooms          13580 non-null  int64         
 3   Type           13580 non-null  object        
 4   Price          13580 non-null  float64       
 5   Method         13580 non-null  object        
 6   SellerG        13580 non-null  object        
 7   Date           13580 non-null  datetime64[ns]
 8   Distance       13580 non-null  float64       
 9   Postcode       13580 non-null  float64       
 10  Bedroom2       13580 non-null  int64         
 11  Bathroom       13580 non-null  int64         
 12  Car            13518 non-null  float64       
 13  Landsize       13580 non-null  float64       
 14  BuildingArea   7130 non-null   float64       
 15  YearBuilt      8205

In [8]:
melb_data.describe()

,Rooms,Price,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Latitude,Longitude,Propertycount,Month
count,13580.000000,1.358000e+04,13580.000000,13580.000000,13580.000000,13580.000000,13518.000000,13580.000000,7130.000000,8205.000000,13580.000000,13580.000000,13580.000000,13580.000000
mean,2.937997,1.075684e+06,10.137776,3105.301915,2.914728,1.534242,1.610075,558.416127,151.967650,1964.684217,-37.809203,144.995216,7454.417378,7.052283
std,0.955748,6.393107e+05,5.868725,90.676964,0.965921,0.691712,0.962634,3990.669241,541.014538,37.273762,0.079260,0.103916,4378.581772,2.536034
min,1.000000,8.500000e+04,0.000000,3000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1196.000000,-38.182550,144.431810,249.000000,1.000000
25%,2.000000,6.500000e+05,6.100000,3044.000000,2.000000,1.000000,1.000000,177.000000,93.000000,1940.000000,-37.856822,144.929600,4380.000000,5.000000
50%,3.000000,9.030000e+05,9.200000,3084.000000,3.000000,1.000000,2.000000,440.000000,126.000000,1970.000000,-37.802355,145.000100,6555.000000,7.000000
75%,3.000000,1.330000e+06,13.000000,3148.000000,3.000000,2.000000,2.000000,651.000000,174.000000,1999.000000,-37.756400,145.058305,10331.000000,9.000000
max,10.000000,9.000000e+06,48.100000,3977.000000,20.000000,8.000000,10.000000,433014.000000,44515.000000,2018.000000,-37.408530,145.526350,21650.000000,12.000000
